## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package and ROS.

### Activate GPU (**Colab only**)

When in Colab, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

In [2]:
try:
    import eagerx_tutorials
except ImportError:
    !{"echo 'Installing eagerx-tutorials with pip.' && pip install eagerx-tutorials >> /tmp/eagerx_install.txt 2>&1"}
    !{"echo 'Setting up virtual display for visualisation' && apt-get install ffmpeg >> /tmp/ffmpeg.txt 2>&1"}

# Make sure ffmpeg is available, else install it.
import subprocess
try:
    subprocess.check_output(['which', 'ffmpeg'])
except Exception as e:
    !{"echo 'Setting up virtual display for visualisation' && apt-get install ffmpeg >> /tmp/ffmpeg.txt 2>&1"}
    
# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()

# Import eagerx
import eagerx
eagerx.set_log_level(eagerx.WARN)

Not running on CoLab.


# EAGERx Advanced usage

EAGERx: https://github.com/eager-dev/eagerx

Documentation: https://eagerx.readthedocs.io/en/master/


## Introduction

In this notebook, you will learn to use eagerx to create a gym-compatible environment. This tutorial covers:
- how to initialize a robot (Go 1 Quadruped Robot).
- how to add pre-processing nodes (i.e. low-level controllers).
- how to fine-tune low-level controllres to achieve the desired behavior.
- how to (de)select various sensors to investigate its effect on the learning performance.

In the remainder of this tutorial, we will go more into detail on these concepts.

Furthermore, you will be asked to add/modify a couple of lines of code, which are marked by

```python

# YOUR CODE HERE

# END OF YOUR CODE
```

## Quadruped Locomotion

In this tutorial, we will learn small offsets to the trot of a [Go 1 Quadruped](https://tribotix.com/product/go1-quadruped-robot/) so that the quadruped makes a left-turn instead of going straight ahead. For this, we will start with an open-loop trot gait that uses a central pattern generator ([CPG](https://en.wikipedia.org/wiki/Central_pattern_generator#Locomotion)) to walk straight ahead. Central pattern generators have been used to control the motion of individual joints of walking robots and in this tutorial we will use one that is based on the Hopf oscillator.

In a nutshell, we will use a Hopf oscillator to produce desired Cartesian (x,z)-coordinates for the quadruped's feet. Then, the Cartesian coordinates are mapped to desired joint positions using the forward kinematics of the quadruped. The desired joint positions are then tracked using the quadruped's joint controllers. Later-on, we will use reinforcement learning to make the agent learn offsets in the y-direction, so that the quadruped makes a left-turn.

<img src="../figures/cpg_icra_simple.png" width=500/>

### Credits for CPG code:

CPG in polar coordinates based on:
Pattern generators with sensory feedback for the control of quadruped
authors: L. Righetti, A. Ijspeert
https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=4543306
Original author: Guillaume Bellegarda

## Let's get started!
Next, we initialize a [Go 1 Quadruped](https://tribotix.com/product/go1-quadruped-robot/) that we prepared for this tutorial.

It has various sensors that run at 20 Hz:
- `joint_position`: the position of the quadruped's leg joints in radians, so 4 legs with 2 joints each.
- `joint_velocity`: the angular velocity in radians of the quadruped's leg joints.
- `force_torque`: The joint reaction forces of the quadruped's leg joints (Fx, Fy, Fz, Mx, My, Mz).
- `orientation`: the orientation of the quadruped's body in quaternions (x, y, z, w).
- `position`: the global Cartesian position of the quadruped's body in meter (x, y, z).
- `velocity`: the global Cartesian velocity of the quadruped's body in meters per second (dx, dy, dz).

and an actuator that runs at 200 Hz:
- `joint_control`: Tracks the desired joint positions of the quadruped's leg joints in radians.

In [3]:
# Available sensors
sensors = ["joint_position", "joint_velocity", "force_torque", "orientation", "position", "velocity"]
actuators = ["joint_control"]

# Create the GO 1 quadruped
from eagerx_tutorials.quadruped.object import Quadruped
robot = Quadruped.make("quadruped", actuators=actuators, sensors=sensors, rate=20)

# Set the quadruped's control rate to 200 Hz.
robot.actuators.joint_control.rate = 200

pybullet build time: Apr 26 2022 03:12:14


Then, we initialize the central pattern generator and Cartesian control nodes.

In [4]:
# Create central pattern generator (uses Hopf Oscillator)
from eagerx_tutorials.quadruped.cpg_gait import CpgGait
cpg = CpgGait.make("cpg", rate=200, gait="TROT", omega_swing=16 * 3.14, omega_stance=4 * 3.14)

# Create Cartesian control (uses the quadruped's forward kinematics)
from eagerx_tutorials.quadruped.cartesian_control import CartesiandPDController
cartesian_control = CartesiandPDController.make("cartesian_control", rate=200)

We add the robot and the two nodes to an empty graph.

Then, we connect the output of the central pattern generator (i.e. Cartesian feet positions) to the Cartesian controller, that in turn, maps them to desired joint positions that are passed to the quadruped's joint controller.

In [5]:
# Initialize empty graph
graph = eagerx.Graph.create([robot, cartesian_control, cpg])

# Interconnect the nodes that results in an initial trot (that moves straight ahead).
graph.connect(source=cpg.outputs.cartesian_pos, target=cartesian_control.inputs.cartesian_pos)
graph.connect(source=cartesian_control.outputs.joint_pos, target=robot.actuators.joint_control)

This open-loop control strategy produces in the nominal case a **forward trot** as demonstrated by the two quadrupeds below:

<img src="../figures/quad.gif"/>

However, we want the quadruped to make a **left-turn** instead!


Therefore, we will define an environment action called `offset`. This offset will be added to the generated open-loop feet pattern.

The offset action range (i.e. high & low) are chose to be relatively small (~1cm). Nonetheless, this should be enough to modify the pattern such that it makes the quadruped turn.


In [6]:
graph.connect(action="offset", target=cpg.inputs.offset)

Then, we connect (a subset of) the sensors as observations in the graph that we would like the agent to use for learning.

In [7]:
# Select the sensors that are to be used as observations
graph.connect(observation="joint_position", source=robot.sensors.joint_position)
graph.connect(observation="joint_velocity", source=robot.sensors.joint_velocity)
graph.connect(observation="force_torque", source=robot.sensors.force_torque)
graph.connect(observation="velocity", source=robot.sensors.velocity)
graph.connect(observation="orientation", source=robot.sensors.orientation, window=2)

# The open-loop pattern is probably also informative to determine relevant offsets.
graph.connect(observation="xs_zs", source=cpg.outputs.xs_zs, skip=True)

We will also visualize the quadruped's movement during training. To reduce the computational overhead of rendering 3D images on the training speed, we will render the quadruped's (x,y) Cartesian coordinates as a proxy instead.

For this, we have already prepared the `XyPlane` node.

In [8]:
# Create xy-plane render node
from eagerx_tutorials.quadruped.overlay import XyPlane
xy_plane = XyPlane.make("xy_plane", rate=5)

# Add node to graph
graph.add(xy_plane)

# The node renders images, based on the x,y position sensor measurements.
graph.connect(source=robot.sensors.position, target=xy_plane.inputs.position)

# Select the output of the node for rendering.
# Can be commented out for faster training
graph.render(xy_plane.outputs.image, rate=5)

It is also possible to inspect the graph using the `eagerx-gui` package.

It can be installed as follows:
```bash
pip3 install eagerx-gui
```
Jupyter notebooks have limited support for interactive applications, so we cannot open the GUI here.
But if we were to run
```python
graph.gui()
```
If we select all the quadruped's sensors as observations, the gui would show the following:

<img src="../figures/tutorial_2_gui.svg" width=720 />

Here we see that the actions of the agent are outputs of *env/actions* and that the observations of the agent are inputs of *env/observations*.
Also, the image output connected to *env/render* will be rendered.
Note that *env/actions*, *env/observations* and *env/render* are connections of the `Graph` to the environment.
They are split up in the GUI as nodes for visualization purposes.

## Environment definition
Then, we define the environment that takes the graph and pybullet engine as input. The actions and observations of the environment are the ones we previously registered in the graph.


### **Exercise 1**
In this exercise, we are going to finish `QuadrupedEnv.step(action)` Specifically, we need to define a reward function that promotes offsets that make the quadruped's trot turn left. We will do so by specifying a reward function:
- Estimate the current `yaw_rate` using the last two yaw sensor measurements. *hint: don't forget to multiply the difference with the sensor rate (`=20 Hz`)*.
- Calculate the `desired_yaw_rate` (e.g. ~20 degrees) and convert it to radians. *hint: use `np.deg2rad(deg)` to convert degrees to radians.*
- Define reward function as the negative squared error between the desired and actual yaw rate.
- Add a little positive *alive* bonus (e.g. 0.25) to the reward, so that the quadruped does not fall down.


In [9]:
# Define Gym Environment
import pybullet
import numpy as np
import gym
from typing import Dict, Tuple

class QuadrupedEnv(eagerx.BaseEnv):
    def __init__(self, name, rate, graph, engine, episode_timeout):
        # Make the backend specification
        from eagerx.backends.single_process import SingleProcess
        backend = SingleProcess.make()
        
        super().__init__(name, rate, graph, engine, backend, force_start=True)
        
        self.steps = None
        self.timeout_steps = int(episode_timeout * rate)
        self.rate = rate  # [Hz] Sensor rate

    @property
    def observation_space(self) -> gym.spaces.Dict:
        """The Space object corresponding to valid observations.

        Per default, the observation space of all registered observations in the graph is used.
        """
        return self._observation_space

    @property
    def action_space(self) -> gym.spaces.Dict:
        """The Space object corresponding to valid actions

        Per default, the action space of all registered actions in the graph is used.
        """
        return self._action_space

    def reset(self):
        """A method that resets the environment to an initial state and returns an initial observation."""
        # Reset number of steps
        self.steps = 0

        # Sample desired states
        states = self.state_space.sample()

        # Perform reset
        obs = self._reset(states)
        
        # Set initial observation for skipped connection 'xs_zs`
        if "xs_zs" in obs:
            obs["xs_zs"][0][:] = [-0.01354526, -0.26941818, 0.0552178, -0.25434446]
        return obs

    def step(self, action: Dict) -> Tuple[Dict, float, bool, Dict]:
        """A method that runs one timestep of the environment's dynamics."""
        
        # Here, we apply a step (i.e. we step the graph dynamics).
        # It returns a dict containing measurements of all registered observations.
        obs = self._step(action)
        self.steps += 1

        # We have access to the last two orientation sensor measurements, 
        # because we used window=2 when connecting `orientation` as an observation in the graph.
        _, _, prev_yaw = pybullet.getEulerFromQuaternion(obs["orientation"][-2])
        roll, pitch, yaw = pybullet.getEulerFromQuaternion(obs["orientation"][-1])
        
        # YOUR CODE HERE
        # 1. Calculate the yaw rate using prev_yaw and yaw (don't forget to scale with self.rate).
        # 2. Calculate the desired yaw_rate (20 degrees) in radians.
        # 3. Calculate the negative squared error between the desired and actual yaw rate.
        # 4. Add a little alive bonus to promote not falling down.
        yaw_rate = (yaw - prev_yaw) * self.rate
        desired_yaw_rate = np.deg2rad(20) # [rad] Desired yaw rate
        yaw_cost = (yaw_rate - desired_yaw_rate) ** 2
        alive_bonus = 0.25
        reward = alive_bonus - yaw_cost
        # END OF YOUR CODE

        # Determine termination condition
        has_fallen = abs(np.rad2deg(roll)) > 40 or abs(np.rad2deg(pitch)) > 40
        timeout = self.steps >= self.timeout_steps

        # Determine done flag
        done = timeout or has_fallen
        
        # Set info about episode truncation
        info = {"TimeLimit.truncated": timeout and not has_fallen}
        return obs, reward, done, info

# Define the pybullet engine
from eagerx_pybullet.engine import PybulletEngine
engine = PybulletEngine.make(rate=200, gui=False, egl=False, process=eagerx.ENVIRONMENT)    

# Initialize Environment
episode_timeout = 10  # [s] number of seconds before timing-out an episode.
env = QuadrupedEnv(name="QuadEnv", rate=20, graph=graph, engine=engine, episode_timeout=episode_timeout)

[WARN]: Adding states to engine nodes can potentially make the environment engine-specific.
[WARN]: Backend 'SINGLE_PROCESS' does not support multiprocessing, so all nodes are launched in the ENVIRONMENT process.


## Training

Once the reward function is implemented, we can start training. We will use the [TQC](https://sb3-contrib.readthedocs.io/en/master/modules/tqc.html) from the `sb3-contrib` package.

You should see a (x,y)-Cartesian coordinate plots while training progresses. After roughly 20-30 episodes of training, the plot should look similar to the one below.

<img src="../figures/train_eps_3_pybullet.gif"/> <img src="../figures/train_eps_3.gif"/>

**Left:** The orignal 3D image that is rendered with pybullet. **Right:** The corresponding (x,y)-Cartesian coordinates of the quadruped's body.

In [10]:
# Stable-baselines
from sb3_contrib import TQC
from eagerx.wrappers import Flatten

# Define hyper parameters for the TQC policy.
hyperparams = dict(
    learning_rate=1e-3,
    tau=0.02,
    gamma=0.98,
    buffer_size=300000,
    learning_starts=100,
    use_sde=True,
    use_sde_at_warmup=True,
    train_freq=8,
    gradient_steps=10,
    verbose=1,
    top_quantiles_to_drop_per_net=0,
    policy_kwargs=dict(n_critics=1, net_arch=dict(pi=[64, 64], qf=[64, 64])),
)

# Initialize the model
model = TQC("MlpPolicy", Flatten(env), **hyperparams)

# Train for 30 episodes
train_episodes = 30
try:
    train_steps = int(train_episodes * episode_timeout * 20)
    # Render top-view of the quadruped's movement
    env.render("human")
    # Start training!
    model.learn(train_steps)
    # Save the final policy
    model.save("last_policy")
except KeyboardInterrupt:
    model.save("last_policy")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
argv[0]=
argv[0]=
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | 3.81     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 22       |
|    time_elapsed    | 36       |
|    total_timesteps | 800      |
| train/             |          |
|    actor_loss      | -20.9    |
|    critic_loss     | 0.408    |
|    ent_coef        | 0.406    |
|    ent_coef_loss   | -5.92    |
|    learning_rate   | 0.001    |
|    n_updates       | 870      |
|    std             | 0.0503   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | 7.38     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 21       |
|    time_elapsed    | 74       |
|  

## Evaluation

Once we have a trained policy, we will evaluate it for a longer period (ie 40 seconds) to verify that we learned the quadruped to trot a circle.

<img src="../figures/eval_eps_4.gif"/>

**Figure**: Example evaluation of a trot that takes a left turn.

In [11]:
# Evaluate
from eagerx_tutorials.quadruped.evaluate import EvaluateEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Load last policy
model = TQC.load("last_policy")

# Create an evaluation environment (renders 3D images).
eval_env = EvaluateEnv(env, engine, episode_timeout=40, render="pybullet")
eval_env.render("human")

# Evaluate policy
helper.evaluate(model, Flatten(eval_env), episode_length=int(40*20), video_rate=20, video_prefix="3d_eval", n_eval_episodes=2)

# Create an evaluation environment (renders xy-plane).
eval_env = EvaluateEnv(env, engine, episode_timeout=40, render="xy-plane")
eval_env.render("human")

# Evaluate policy
helper.evaluate(model, Flatten(eval_env), episode_length=int(40*20), video_rate=20, video_prefix="xy_eval", n_eval_episodes=2)

argv[0]=
argv[0]=
Start evaluation episode 0 of 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 800/800 [00:47<00:00, 16.81it/s]


Start video writer
Showing episode 0 with episodic reward: 0.0


Start evaluation episode 1 of 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 800/800 [00:48<00:00, 16.50it/s]


...

Start video writer
Showing episode 0 with episodic reward: 0.0


Start evaluation episode 1 of 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 800/800 [00:27<00:00, 28.61it/s]


Start video writer
Showing episode 1 with episodic reward: 0.0


Finished evaluation with mean episodic reward: 0.0


## Exercise 2

Currently, we have added most of the quadruped's sensors as observation to the environment for simplicity. However, not all sensors are relevant/necessary for learning. For example, the policy should be invariant to the global Cartesian coordinates of the quadruped. Therefore, adding the `position` sensor as an observation to the environment does not make much sense.

Find out which sensors are essential for learning to trot a circle.
- You can (de)select sensors as observations by (un)commenting the corresponding `graph.connect(...)` lines.
- Prune the sensors that do not influence the final performance.